In [1]:
import time

In [2]:
import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"


import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scanpy as sc
import anndata as ad

import logging
from experiments_utils import load_scvi_model_if_exists

import matplotlib.pyplot as plt

%matplotlib inline
import torch.nn as nn
import torch
import logging
import matplotlib.pyplot as plt

from scvi.dataset import GeneExpressionDataset
from scvi.inference import UnsupervisedTrainer
from scvi.models import VAE, SCSphereFull

from experiments_utils import get_eb_full


N_EPOCHS = 300

/data/yosef2/users/pierreboyeau/conda/envs/DE/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9002. The TBB threading layer is disabled.


In [5]:
data_path = "/data"
save_path = os.path.join(data_path, "annds_sparse")

In [6]:
ann_paths = os.listdir(save_path)
ann_paths = [os.path.join(save_path, path) for path in ann_paths]

In [7]:
do_sph_deep = True

mdl_sph_kwargs = dict(
#     n_genes=dataset.nb_genes,
#     n_batches=dataset.n_batches,
    n_latent=11,
    do_depth_reg=do_sph_deep,
    constant_pxr=True,
    cell_specific_px=False,
    scale_norm="softmax",
    library_nn=True,
    dropout_rate=0.0,
    deep_architecture=do_sph_deep,
)
trainer_sph_kwargs = dict(
    train_library=True,
    k=25,
    loss_type="IWELBO",
    batch_size=1024,
    weight_decay=1e-4,
    optimizer_type="adam",
    test_indices=[],
)

In [8]:
mdl_iw_kwargs = dict(
    n_latent=10,
    dropout_rate=0.0,
    decoder_dropout_rate=0.0,
    reconstruction_loss="nb",
    dispersion="gene",
    n_layers=1,
    use_batch_norm=False,
    use_weight_norm=False,
    use_layer_norm=True,
    with_activation=nn.ReLU(),
)
trainer_iw_kwargs = dict(
    train_library=True,
    k=25,
    batch_size=1024,
    weight_decay=1e-4,
    loss_type="IWELBO",
    optimizer_type="adam",
    test_indices=[],
)

In [13]:
# best_genes_smartseq2 = best_genes.copy()

In [15]:
pairs = [
    ("b", "monocyte"),
    ("b", "dc"),
    ("b", "pdc"),
    ("monocyte", "pdc"),
    ("dc", "pdc"),
    ("monocyte", "dc"),
]

In [16]:
def get_preds(idx_a, idx_b, trainer_iw):  
    if len(idx_a) >= 1000:
        idx_a = np.random.choice(idx_a, size=np.minimum(len(idx_a), 1000), replace=False)
    if len(idx_b) >= 1000:
        idx_b = np.random.choice(idx_b, size=np.minimum(len(idx_b), 1000), replace=False)
    iw_props_eb = get_eb_full(
        trainer=trainer_iw,
        idx_a=idx_a,
        idx_b=idx_b,
        offset=None,
        delta=None,
        filter_cts=True,
#         coef_truncate=0.5,
        n_samples=30000,
        do_batch_specific="separation",
        posterior_chunks=200,
    )

    res_loop = (
        pd.DataFrame(iw_props_eb)
        .assign(
#             lfc_gt=metadatab.logFC.values,
#             padj=metadatab["adj.P.Val"].values,
#             pval=metadatab["P.Value"].values,
#             GS=final_ann.var.index,
        )
    )
    return res_loop

## Quantitative analysis

In [22]:
ann_paths_arr = np.array(ann_paths) #[[0, 3, 5, 6]]
ann_paths_arr

array(['/data/yosef2/users/pierreboyeau/data/PbmcBench/Inter-dataset/PbmcBench/annds_sparse/Drop-Seq_annd.h5ad',
       '/data/yosef2/users/pierreboyeau/data/PbmcBench/Inter-dataset/PbmcBench/annds_sparse/Seq-Well_annd.h5ad',
       '/data/yosef2/users/pierreboyeau/data/PbmcBench/Inter-dataset/PbmcBench/annds_sparse/10Xv2_annd.h5ad',
       '/data/yosef2/users/pierreboyeau/data/PbmcBench/Inter-dataset/PbmcBench/annds_sparse/10Xv3_annd.h5ad',
       '/data/yosef2/users/pierreboyeau/data/PbmcBench/Inter-dataset/PbmcBench/annds_sparse/inDrop_annd.h5ad',
       '/data/yosef2/users/pierreboyeau/data/PbmcBench/Inter-dataset/PbmcBench/annds_sparse/Smart-Seq2_annd.h5ad',
       '/data/yosef2/users/pierreboyeau/data/PbmcBench/Inter-dataset/PbmcBench/annds_sparse/CEL-Seq_annd.h5ad'],
      dtype='<U104')

In [23]:
import itertools

groups = []
for r in np.arange(1, 6):
# for r in [2, 3, 4]:
    all_gps = list(itertools.combinations(ann_paths_arr, r))
    all_gps = [list(gp) for gp in all_gps]
    print(len(all_gps))
    groups += all_gps
    

7
21
35
35
21


### Run experiment per se

In [25]:
# n_total = len(groups[-1])
n_total = 7

In [26]:
n_total

7

In [27]:
pairs = [
    ("b", "monocyte"),
    ("b", "dc"),
    ("b", "pdc"),
    ("monocyte", "pdc"),
    ("dc", "pdc"),
    ("monocyte", "dc"),
]

##### Baselines

In [28]:
gp = groups[30]
sub_ann_paths = gp

In [29]:
experiment_path = "./exp/pbmc_multibatch_b"


In [30]:
pairs = [
    ("b", "monocyte"),
    ("b", "dc"),
    ("b", "pdc"),
    ("monocyte", "pdc"),
    ("dc", "pdc"),
    ("monocyte", "dc"),
]

In [31]:
import rpy2.robjects as ro

In [32]:
ro.r["library"]("BiocParallel")
ro.r("BiocParallel::register(BiocParallel::MulticoreParam(workers=40))")

MulticoreParam,[RTYPES.S4SXP]
SnowParam,[RTYPES.S4SXP]
SerialParam,[RTYPES.S4SXP]


In [33]:
def filter_indices(my_indices, my_batches):
    new_indices = []
    unique_batches = np.unique(my_batches)
    for batch in unique_batches:
        _indices = my_indices[my_batches == batch]
        if _indices.shape[0] >= 500:
            _indices = np.random.choice(_indices, size=500, replace=True)
        new_indices += list(_indices)
    new_indices = np.array(new_indices)
    return new_indices

In [34]:
def loop_procedure_baselines(sub_ann_paths):
    annds = []
    ann_names = []
    for ann_path in sub_ann_paths:
        ann_name = ann_path.split("/")[-1].split("_")[0]
        ann_names.append(ann_name)
    ann_names_pad = ann_names + (n_total - len(ann_names)) * [None]
    print(ann_names_pad)
    final_ann = adata[adata.obs.batch.isin(ann_names)]

    tagger = "_".join(ann_names)
    iw_filename = os.path.join(experiment_path, "{}_genes_from_full.pt".format(tagger))
    print(iw_filename)
    print("Dataset properties")
    print(final_ann)

    batch_indices, batch_cats = pd.factorize(final_ann.obs.final_batch)
    lbl_indices, lbl_cats = pd.factorize(final_ann.obs.x)
    dataset = GeneExpressionDataset()
    dataset.populate_from_data(
        X=np.asarray(final_ann.X.astype(float).todense()), 
        batch_indices=batch_indices.astype(float),
        labels=lbl_indices,
        cell_types=lbl_cats
    )
    print(dataset.X.min(), dataset.X.max())
    ## B / Monocytes
    np.random.seed(0)
    name_to_idx = {
        "b": np.where(final_ann.obs.x == "B cell")[0], 
        "monocyte": np.where(final_ann.obs.x.str.contains("monocyte"))[0],
        "dc": np.where(final_ann.obs.x == "Dendritic cell")[0],
        "pdc": np.where(final_ann.obs.x == "Plasmacytoid dendritic cell")[0],
    }
    res_loop_full = pd.DataFrame()
    for label_a, label_b in pairs:
        idx_a = name_to_idx[label_a]
        idx_b = name_to_idx[label_b]
        if (len(idx_a) == 0) or (len(idx_b) == 0):
            continue
        batches_a = dataset.batch_indices[idx_a].squeeze()
        batches_b = dataset.batch_indices[idx_b].squeeze()
        ba = np.unique(batches_a)
        bb = np.unique(batches_b)
        if len(np.intersect1d(ba, bb)) == 0:
            continue

        from R_interop import all_predictionsB

        idx_a = filter_indices(idx_a, batches_a)
        idx_b = filter_indices(idx_b, batches_b)

        TEST_INDICES = np.concatenate([idx_a, idx_b])
        print(TEST_INDICES.shape)

        x_test = dataset.X[TEST_INDICES, :]
        y_test = np.array([0] * len(idx_a) + [1] * len(idx_b))
        batch_test = dataset.batch_indices[TEST_INDICES, :].squeeze()
        x_test[:, -1] = 1.0 + x_test[:, -1]

        DIR_PATH = "exp/pbmc_multibatch_c/"
        # os.makedirs(DIR_PATH)
        filename = "temp_experiment.pickle"

        delta = 0.3
        PATH_TO_SCRIPTS = "MYPATHTOSCRIPTS"

        data_path = os.path.join(DIR_PATH, "data.npy")
        labels_path = os.path.join(DIR_PATH, "labels.npy")
        batches_path = os.path.join(DIR_PATH, "batch_indices.npy")

        np.save(data_path, x_test.squeeze().astype(int))
        np.savetxt(labels_path, y_test.squeeze())
        np.savetxt(batches_path, batch_test.squeeze())
        try:
            other_predictions = all_predictionsB(
                filename=filename,
                data_path=data_path,
                labels_path=labels_path,
                path_to_scripts=PATH_TO_SCRIPTS,
                lfc_threshold=delta,
                batches=batches_path,
                do_edger=False
            )
        except:
            continue

        df_other = pd.DataFrame()
        for _, row in other_predictions.iterrows():
            vals = row.to_dict()
            vals = {**dict(GS=final_ann.var.index), **row.to_dict()}

            dfi = (
                pd.DataFrame(vals)
            )
            df_other = df_other.append(dfi)
        df_other = df_other.assign(
            label_a=label_a, 
            label_b=label_b,
            n_batches=len(sub_ann_paths),
            batch0=ann_names_pad[0],
            batch1=ann_names_pad[1],
            batch2=ann_names_pad[2],
            batch3=ann_names_pad[3],
            batch4=ann_names_pad[4],
            batch5=ann_names_pad[5],
            batch6=ann_names_pad[6],
            nb=len(np.intersect1d(ba, bb))
        )
        res_loop_full = res_loop_full.append(df_other)
    return res_loop_full

In [35]:
experiment_path = "./exp/pbmc_multibatch_baselinesPROPER_SUBFILTER_part2_fourplus"

os.makedirs(experiment_path)

# df_path = os.path.join(experiment_path, "results_save_quantitative.pickle")
df_path = os.path.join(experiment_path, "results_save_quantitative_genes_from_full2.pickle")

In [36]:
from tqdm.auto import tqdm

In [37]:
change_d = dict(
    label_a="category",
    label_b="category",
    GS="category",
    batch0="category",
    batch1="category",
    batch2="category",
    batch3="category",
    batch4="category",
    batch5="category",
    batch6="category",
    n_batches=np.int16,
#     nb=np.int16,
)

In [38]:
# [idx for idx, gpi in enumerate(groups) if gpi == gp]

In [ ]:
res_qt = pd.DataFrame()
for gp in tqdm(groups[63:]):
    res_ = loop_procedure_baselines(gp)
    res_qt = res_qt.append(res_).astype(change_d)
    res_qt.to_pickle(df_path)
#     break

  0%|          | 0/56 [00:00<?, ?it/s]

['Drop-Seq', 'Seq-Well', '10Xv2', '10Xv3', None, None, None]
./exp/pbmc_multibatch_baselinesPROPER_SUBFILTER_part2_fourplus/Drop-Seq_Seq-Well_10Xv2_10Xv3_genes_from_full.pt
Dataset properties
View of AnnData object with n_obs × n_vars = 77689 × 3000
    obs: 'x', 'batch', 'final_batch', 'batchb', 'annotation', 'transfer_rf', 'transfer_boost'
    var: 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg'


INFO:scvi.dataset.dataset:Remapping labels to [0,N]
INFO:scvi.dataset.dataset:Remapping batch_indices to [0,N]


0.0 34710.0
(4000,)





                                                                              
Done!





                                                                              
Done!









-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)





...


out of 2841 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 249, 8.8%
LFC < 0 (down)     : 29, 1%
outliers [1]       : 0, 0%
low counts [2]     : 2142, 75%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results









   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.



-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)






out of 2841 with nonzero total read count
adjusted p-value < 0.05
LFC > 0.50 (up)    : 144, 5.1%
LFC < -0.50 (down) : 16, 0.56%
outliers [1]       : 0, 0%
low counts [2]     : 2416, 85%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



[1] "condtX1"
[1] "(Intercept)" "condtX1"     "batchX1"     "batchX2"    
(2627,)





                                                                              
Done!







In [ ]:
res_loop

##### DGMs

In [56]:
def loop_procedure(sub_ann_paths):
    annds = []
    ann_names = []
    for ann_path in sub_ann_paths:
        ann_name = ann_path.split("/")[-1].split("_")[0]
        ann_names.append(ann_name)
    ann_names_pad = ann_names + (n_total - len(ann_names)) * [None]
    print(ann_names_pad)
    final_ann = adata[adata.obs.batch.isin(ann_names)]    
    
    tagger = "_".join(ann_names)
    iw_filename = os.path.join(experiment_path, "{}_genes_from_full.pt".format(tagger))
    sph_filename = os.path.join(experiment_path, "{}_genes_from_full_sph.pt".format(tagger))
    print(iw_filename)
    
    print("Dataset properties")
    print(final_ann)
    
    batch_indices, batch_cats = pd.factorize(final_ann.obs.final_batch)
    lbl_indices, lbl_cats = pd.factorize(final_ann.obs.x)
    dataset = GeneExpressionDataset()
    dataset.populate_from_data(
        X=np.asarray(final_ann.X.astype(float).todense()), 
        batch_indices=batch_indices.astype(float),
        labels=lbl_indices,
        cell_types=lbl_cats
    )
    print(dataset.X.min(), dataset.X.max())

    #################
    # SCPHERE
    sph_start = time.time()
    mdl_sph = SCSphereFull(n_genes=dataset.nb_genes, n_batches=dataset.n_batches, **mdl_sph_kwargs)
    trainer_sph = UnsupervisedTrainer(
        model=mdl_sph, gene_dataset=dataset, **trainer_sph_kwargs
    )
    lr_sph = trainer_sph.find_lr()
    logging.info("Using learning rate {}".format(lr_sph))
    mdl_sph = SCSphereFull(n_genes=dataset.nb_genes, n_batches=dataset.n_batches, **mdl_sph_kwargs)
    trainer_sph = UnsupervisedTrainer(
        model=mdl_sph, gene_dataset=dataset, **trainer_sph_kwargs
    )
    trainer_sph.train(n_epochs=250, lr=lr_sph)
    train_sph_time = time.time() - sph_start
    mdl_sph.eval()
    # Empirical experiments show that without dropout regularization, models will overfit in terms of reconstruction error after a while
    # eval mode does not work as well as train mode

    torch.save(mdl_sph.state_dict(), sph_filename)

    #######################
    
    # IW model
    iw_start = time.time()
    mdl_iw = VAE(n_input=dataset.nb_genes, n_batch=dataset.n_batches, **mdl_iw_kwargs)
    trainer_iw = UnsupervisedTrainer(
        model=mdl_iw, gene_dataset=dataset, **trainer_iw_kwargs
    )
    lr_iw = trainer_iw.find_lr()
    mdl_iw = VAE(n_input=dataset.nb_genes, n_batch=dataset.n_batches, **mdl_iw_kwargs)
    trainer_iw = UnsupervisedTrainer(
        model=mdl_iw, gene_dataset=dataset, **trainer_iw_kwargs
    )
    mdl_iw, train_iw = load_scvi_model_if_exists(mdl_iw, filename=iw_filename)
    if train_iw:
        logging.info("Using learning rate {}".format(lr_iw))
        trainer_iw.train(n_epochs=250, lr=lr_iw)
    train_iw_time = time.time() - iw_start
    mdl_iw.eval()
    torch.save(mdl_iw.state_dict(), iw_filename)
    
    # SCPHERE
    
    
    ## B / Monocytes
    np.random.seed(0)
    name_to_idx = {
        "b": np.where(final_ann.obs.x == "B cell")[0], 
        "monocyte": np.where(final_ann.obs.x.str.contains("monocyte"))[0],
        "dc": np.where(final_ann.obs.x == "Dendritic cell")[0],
        "pdc": np.where(final_ann.obs.x == "Plasmacytoid dendritic cell")[0],
    }
    res_loop_full = pd.DataFrame()
    for label_a, label_b in pairs:
        idx_a = name_to_idx[label_a]
        idx_b = name_to_idx[label_b]
        if (len(idx_a) == 0) or (len(idx_b) == 0):
            continue
        ba = np.unique(dataset.batch_indices[idx_a].squeeze())
        bb = np.unique(dataset.batch_indices[idx_b].squeeze())
        if len(np.intersect1d(ba, bb)) == 0:
            continue
        cell_type_start = time.time()
        res_loop_iw = (
            get_preds(idx_a, idx_b, trainer_iw)
            .assign(
                label_a=label_a, 
                label_b=label_b,
                GS=final_ann.var.index,
                n_batches=len(sub_ann_paths),
                batch0=ann_names_pad[0],
                batch1=ann_names_pad[1],
                batch2=ann_names_pad[2],
                batch3=ann_names_pad[3],
                batch4=ann_names_pad[4],
                batch5=ann_names_pad[5],
                batch6=ann_names_pad[6],
                nb=len(np.intersect1d(ba, bb)),
                train_time=train_iw_time,
                algorithm="scvi-lvm",
            )
        )
        res_loop_full = res_loop_full.append(res_loop_iw.assign(inference_time=time.time() - cell_type_start))

        cell_type_start = time.time()
        res_loop_sph = (
            get_preds(idx_a, idx_b, trainer_sph)
            .assign(
                label_a=label_a, 
                label_b=label_b,
                GS=final_ann.var.index,
                n_batches=len(sub_ann_paths),
                batch0=ann_names_pad[0],
                batch1=ann_names_pad[1],
                batch2=ann_names_pad[2],
                batch3=ann_names_pad[3],
                batch4=ann_names_pad[4],
                batch5=ann_names_pad[5],
                batch6=ann_names_pad[6],
                nb=len(np.intersect1d(ba, bb)),
                train_time=train_sph_time,
                algorithm="scPhere-lvm",
            )
        )
        res_loop_full = res_loop_full.append(res_loop_sph.assign(inference_time=time.time() - cell_type_start))
    return res_loop_full

In [57]:
experiment_path = "./exp/pbmc_multibatch"

# os.makedirs(experiment_path)

# df_path = os.path.join(experiment_path, "results_save_quantitative.pickle")
df_path = os.path.join(experiment_path, "results_save_quantitative_genes_from_full_with_scphere_30plus.pickle")

In [ ]:
from tqdm.auto import tqdm

In [59]:
change_d = dict(
    label_a="category",
    label_b="category",
    GS="category",
    batch0="category",
    batch1="category",
    batch2="category",
    batch3="category",
    batch4="category",
    batch5="category",
    batch6="category",
    n_batches=np.int16,
#     nb=np.int16,
)

In [60]:
torch.cuda.empty_cache()

In [ ]:
res_qt

In [62]:
# groups

In [63]:
# [idx for idx in range(len(groups)) if groups[idx] == gp]

In [65]:
faulty_gps = []

In [ ]:
res_qt = pd.DataFrame()
for gp in tqdm(groups[30:]):
    try:
        res_ = loop_procedure(gp)
        res_qt = res_qt.append(res_).astype(change_d)
        res_qt.to_pickle(df_path)
    except Exception as e:
        faulty_gps.append(gp)
        print(e)
#         pass
#         break
#     break

  0%|          | 0/89 [00:00<?, ?it/s]

['Drop-Seq', 'Seq-Well', 'inDrop', None, None, None, None]
./exp/pbmc_multibatch/Drop-Seq_Seq-Well_inDrop_genes_from_full.pt
Dataset properties
View of AnnData object with n_obs × n_vars = 56972 × 3000
    obs: 'x', 'batch', 'final_batch', 'batchb', 'annotation', 'transfer_rf', 'transfer_boost'
    var: 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg'


INFO:scvi.dataset.dataset:Remapping labels to [0,N]
INFO:scvi.dataset.dataset:Remapping batch_indices to [0,N]
INFO:root:Using Deep architecture ...
INFO:root:Using deep architecture
INFO:root:Unique optim


0.0 34710.0


INFO:root:Problem with automatic LR choice, using 1e-4 as default
INFO:root:Using learning rate 0.0001
INFO:root:Using Deep architecture ...
INFO:root:Using deep architecture
INFO:root:Unique optim


...

INFO:root:Using 5000 samples for log-evidence estimation ...
INFO:root:... done!
INFO:root:px reweight
INFO:root:Step 3: Compute scales from original batches


torch.Size([1, 294])
torch.Size([10290, 294])
log_pz -24.99059295654297 -9.898506164550781
log_px_zs -4082.760986328125 -71.755859375
log_px -1725.403564453125 -78.61653900146484
log_qz_x -391.15472412109375 5.792505264282227
log_probas -51.568389892578125 3.9127540588378906
tensor([0])


/data/yosef2/users/pierreboyeau/scVI-DE/scvi/inference/posterior.py:402: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
INFO:root:ESS: 212.37276307227356
INFO:root:Filtering observations: Keeping (91,) cells from original 102 sample size
INFO:root:Using 110 posterior samples per cell
INFO:root:Step 1: Getting posterior samples
INFO:root:Step 2: Compute overall importance weights


[ 3194  3196  3197  3198  3199  3200  3201  3202  3203  3204  3205  3206
  3207  3208  3210  3211  3212  3213  3214  3215  3216  3217  3218  3219
  3220  3221  3850  3851  3852  3853  3854  3855  3856  3857  3858  3859
  3860  3861  3862  3863  3864  3865  3866  3867  3868  3869  3870  3871
  3872  3873  3874  3875  6708  6709  6769  6831  6894  6957  6979  7039
  7045  7055  9899  9903 13924 14099 14146 14329 14537 15460 15661 16056
 16063 16782 16793 16868 16925 17049 17241 17521 17765 18256 18571 18642
 18654 18961 19300 19328 19490 19601 19651]
None


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
res_qt

In [ ]:
res_qt.label_a.unique()